In [77]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


file_path = "DATASET.xlsx"

xls = pd.ExcelFile(file_path)
print("✅ Available sheets:", xls.sheet_names)

# Read both sheets
sheet1_df = pd.read_excel(file_path, sheet_name="Sheet1", header=0)
sheet2_df = pd.read_excel(file_path, sheet_name="Sheet2", header=0)
sheet3_df = pd.read_excel(file_path, sheet_name="yearly_rainfall", header=0)

sheet1_df.head(10)    #displaying read sheet1,2,3
print(sheet1_df.tail(10))
print(sheet2_df.head(10))



✅ Available sheets: ['Sheet1', 'Sheet2', 'yearly_rainfall']
    Serial No.     Year        Period Months_Covered  Period_Length  \
61          62  2023-24  Post-Monsoon        May–Jan              9   
62          63  2023-24  Post-Monsoon        May–Jan              9   
63          64  2023-24  Post-Monsoon        May–Jan              9   
64          65  2023-24  Post-Monsoon        May–Jan              9   
65          66  2023-24  Post-Monsoon        May–Jan              9   
66          67  2023-24  Post-Monsoon        May–Jan              9   
67          68  2023-24  Post-Monsoon        May–Jan              9   
68          69  2023-24  Post-Monsoon        May–Jan              9   
69          70  2023-24  Post-Monsoon        May–Jan              9   
70          71  2023-24  Post-Monsoon        May–Jan              9   

            District  No_of_HNS  Rise_<2m  Rise_<2m_%  Rise_2-4m  ...  \
61           Latehar          6         0        0.00          0  ...   
62         L

In [78]:
print(sheet1_df.describe())  #describing df
print(sheet2_df.describe())
print(sheet3_df.describe())

       Serial No.  Period_Length  No_of_HNS   Rise_<2m  Rise_<2m_%  Rise_2-4m  \
count   71.000000      71.000000  71.000000  71.000000   70.000000  71.000000   
mean    36.000000       6.633803  14.450704   1.887324   11.944571   1.676056   
std     20.639767       2.064942   9.804939   2.891116   13.066350   3.421673   
min      1.000000       4.000000   5.000000   0.000000    0.000000   0.000000   
25%     18.500000       4.000000   9.000000   0.000000    0.000000   0.000000   
50%     36.000000       7.000000  11.000000   1.000000    8.010000   0.000000   
75%     53.500000       9.000000  17.500000   2.500000   20.000000   2.000000   
max     71.000000       9.000000  54.000000  19.000000   50.000000  23.000000   

       Rise_2-4m_%   Rise_>4m  Rise_>4m_%   Fall_<2m  Fall_<2m_%  Fall_2-4m  \
count    70.000000  71.000000   70.000000  71.000000   70.000000  71.000000   
mean     12.125000   2.154930   17.201429   3.450704   22.751143   3.436620   
std      18.306066   3.236611   2

In [79]:
sheet1_df=sheet1_df.drop_duplicates()  # deleting duplicates if any
sheet2_df=sheet2_df.drop_duplicates()
sheet3_df=sheet3_df.drop_duplicates()


In [80]:
print(sheet2_df.columns.tolist())
sheet2_df["Date"]=pd.to_datetime(sheet2_df["Date"],errors="coerce")


['Serial No.', 'District', 'Date', 'Actual Rainfall (mm)', 'Normal Rainfall (mm)', 'Departure (mm)']


In [81]:
print(sheet1_df['District'].unique())  # to determine spelling mistake in district columns
print(sheet2_df['District'].unique())

correction={"Devghar":"Deoghar"}
sheet1_df['District']=sheet1_df['District'].replace(correction)



['Bokaro' 'Chatra' 'Deoghar' 'Dhanbad' 'Dumka' 'Garhwa' 'Giridih' 'Godda'
 'Gumla' 'Hazaribag' 'Jamtara' 'Khunti' 'Koderma' 'Latehar' 'Lohardaga'
 'Pakur' 'Palamau' 'Paschim Singbhum' 'Purba Singbhum' 'Ramgarh' 'Ranchi'
 'Sahebganj' 'Saraikela' 'Simdega' 'Sahibganj']
['Ranchi' 'Dhanbad' 'East Singhbhum' 'Bokaro' 'West Singhbhum' 'Chatra'
 'Deoghar' 'Dumka' 'Giridih' 'Godda' 'Gumla' 'Hazaribagh' 'Jamtara'
 'Khunti' 'Koderma' 'Latehar' 'Lohardaga' 'Pakur' 'Palamu' 'Ramgarh'
 'Sahibganj' 'Saraikela' 'Simdega' 'Garhwa']


In [110]:
# check category distributions.String/text data converted to numeric
#columns are district,year,month-covered,period,category etc

# text to numeric using mapping
def clean_df(df):

    for col in df.columns:
        if df[col].dtype in ['int64','float64']:
            df[col]=pd.to_numeric(df[col],errors="coerce")
            df[col]=df[col].fillna(df[col].mean())
            df.loc[df[col]<0,col]=0
            df[col]=df[col].round(2)
        else:
            df[col]=df[col].fillna(df[col].mode()[0])
            df[col]=df[col].astype(str).str.strip().str.lower()
    return df


sheet1_cleaned = clean_df(sheet1_df.copy())
sheet2_cleaned = clean_df(sheet2_df.copy())
sheet3_cleaned = sheet3_df.copy()


year_mapping={
    "2020-21":2021,
    "2021-22":2022,
    "2022-23":2023,
    "2023-24":2024
}

sheet1_cleaned["Year"]=sheet1_cleaned["Year"].map(year_mapping)

period_mapping={
    "pre-monsoon":0,
    "monsoon":1,
    "post-monsoon":2
}

sheet1_cleaned["Period"]=sheet1_cleaned["Period"].map(period_mapping)

#month done through encode since range varies
from sklearn.preprocessing import LabelEncoder

encoder_months=LabelEncoder()
sheet1_cleaned["Months_Covered"]=encoder_months.fit_transform(sheet1_cleaned["Months_Covered"])


combined_districts = pd.concat([
    sheet1_cleaned["District"],
    sheet2_cleaned["District"]
])

encoder = LabelEncoder()
encoder.fit(combined_districts)

# Now encode using the same encoder
sheet1_cleaned["District"] = encoder.transform(sheet1_cleaned["District"])
sheet2_cleaned["District"] = encoder.transform(sheet2_cleaned["District"])

#sheet3
category_mapping={
     "SCANTY":0,
     "DEFICIENT":1,
     "NORMAL":2,
     "EXCESS":3
}
sheet3_cleaned["CATEGORY"]=sheet3_cleaned["CATEGORY"].map(category_mapping)

#year storing as numeric
sheet3_cleaned["YEAR"]=pd.to_numeric(sheet3_df["YEAR"],errors="coerce").astype(int)


sheet3_cleaned=sheet3_cleaned.rename(columns={"ACTUAL (mm)":"Yearly_Actual_mm",
                                    "NORMAL (mm)":"Yearly_Normal_mm",
                                    "DEPARTURE (mm)":"Yearly_Departure_mm",
                                    "CATEGORY":"Yearly_Category"})

print(sheet2_cleaned.head(10))


   District        Date  Actual Rainfall (mm)  Normal Rainfall (mm)  \
0        23  2024-06-01                 342.1                   280   
1         3  2024-06-01                 310.5                   275   
2         5  2024-06-01                 365.2                   290   
3         0  2024-06-01                 298.7                   270   
4        28  2024-06-01                 370.4                   295   
5         1  2024-06-01                 285.6                   265   
6         2  2024-06-01                 295.3                   268   
7         4  2024-06-01                 312.7                   272   
8         7  2024-06-01                 301.9                   270   
9         8  2024-06-01                 280.4                   260   

   Departure (mm)  Year  
0            62.1  2024  
1            35.5  2024  
2            75.2  2024  
3            28.7  2024  
4            75.4  2024  
5            20.6  2024  
6            27.3  2024  
7         

In [84]:
#feature eng columns are added

# --- Sheet1 (Groundwater) ---
sheet1_cleaned["Net_GW_Change"] = sheet1_cleaned["Total_Rise"] - sheet1_cleaned["Total_Fall"]

# --- Sheet2 (Monthly Rainfall) ---
sheet2_cleaned["Rainfall_Anomaly"] = sheet2_cleaned["Actual Rainfall (mm)"] - sheet2_cleaned["Normal Rainfall (mm)"]
sheet2_cleaned["Rainfall_Ratio"] = sheet2_cleaned["Actual Rainfall (mm)"] / (sheet2_cleaned["Normal Rainfall (mm)"] + 1e-5)

# --- Sheet3 (Yearly Rainfall) ---
sheet3_cleaned = sheet3_cleaned.sort_values("YEAR")
sheet3_cleaned["Yearly_Anomaly"] = sheet3_cleaned["Yearly_Actual_mm"] - sheet3_cleaned["Yearly_Normal_mm"]
sheet3_cleaned["Yearly_Ratio"] = sheet3_cleaned["Yearly_Actual_mm"] / (sheet3_cleaned["Yearly_Normal_mm"] + 1e-5)
sheet3_cleaned["Prev_Year_Rainfall"] = sheet3_cleaned["Yearly_Actual_mm"].shift(1)
sheet3_cleaned["Rainfall_3yr_Avg"] = sheet3_cleaned["Yearly_Actual_mm"].rolling(3, min_periods=1).mean()


In [137]:

rainfall_agg = sheet2_cleaned.groupby(["District", "Year"]).agg({
    "Actual Rainfall (mm)": "mean",
    "Normal Rainfall (mm)": "mean",
    "Departure (mm)": "mean"
}).reset_index()


sheet1_cleaned["Year"] = sheet1_cleaned["Year"].astype(int)
rainfall_agg["Year"] = rainfall_agg["Year"].astype(int)

sheet1_cleaned["District"] = sheet1_cleaned["District"].astype(int)
rainfall_agg["District"] = rainfall_agg["District"].astype(int)

# Merge
merged_df = pd.merge(sheet1_cleaned, rainfall_agg, on=["District", "Year"], how="inner")

# View result
print(merged_df.shape)
print(merged_df.head())

# Step 4: Merge
merged_df = pd.merge(sheet1_cleaned, rainfall_agg, on=["District", "Year"], how="inner")


# Step 5: View
print(merged_df.head())

merged_df.to_excel("processed_data.xlsx", index=False)
print("✅ Merge complete. Saved as merged_data.xlsx")



(57, 24)
   Serial No.  Year  Period  Months_Covered  Period_Length  District  \
0           1  2024       0               0              4         0   
1           2  2024       0               0              4         1   
2           3  2024       0               0              4         2   
3           4  2024       0               0              4         3   
4           5  2024       0               0              4         4   

   No_of_HNS  Rise_<2m  Rise_<2m_%  Rise_2-4m  ...  Fall_<2m_%  Fall_2-4m  \
0         14         4       28.57          4  ...        7.14          0   
1         11         5       45.45          2  ...        0.00          1   
2         10         4       40.00          5  ...        0.00          0   
3         17         6       35.29          5  ...        5.88          0   
4          8         1       12.50          5  ...        0.00          0   

   Fall_2-4m_%  Fall_>4m  Fall_>4m_%  Total_Rise  Total_Fall  \
0         0.00         0       

In [ ]:
import pandas as pd

output_file = "processed_data.xlsx"

# sheet3_cleaned appended at merged_data 
with pd.ExcelWriter(output_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    sheet3_cleaned.to_excel(writer, sheet_name="Sheet2", index=False)

print("✅ 'Sheet3_Cleaned' added to 'merged_data.xlsx'")


✅ 'Sheet3_Cleaned' added to 'merged_data.xlsx'
